https://sciencecareer.data.blog/2020/05/04/auto-colorization-of-black-and-white-images-using-machine-learning-auto-encoders-technique/

In [1]:
import os
import glob
import cv2
import time
import random
import webbrowser
import numpy as np
import pandas as pd
from numpy import reshape
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import savefig

import tensorflow as tf
#from tensorflow.keras import datasets, layers, models
import tensorflow.keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Flatten, Reshape
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Flatten, InputLayer, BatchNormalization, Input, RepeatVector, concatenate
from tensorflow.keras.models import Model, Sequential, model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from skimage import color
from skimage.io import imsave
from skimage.transform import resize
from skimage.io import imsave, imread, imshow
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function       #repasar funcion y explicar en exposicion

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Get images
img = []
for filename in os.listdir('color256/'):
    img.append(img_to_array(load_img('color256/'+filename)))

In [4]:
len(img)

6899

In [5]:
X = []
Y = []

for x in img:
    lab = rgb2lab(x)
    X.append(lab[:,:,0])
    Y.append(lab[:,:,1:] / 128)

In [6]:
X = np.asarray(X, dtype=float)/255.0
Y = np.asarray(Y, dtype=float)/255.0

In [8]:
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encoder_input, outputs=decoder_output)

In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)
optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mse' , metrics=['accuracy'])

In [11]:
%%time
history = model.fit(X,Y,validation_split=0.10, epochs=50, callbacks=callback)

Epoch 1/50
195/195 [==============================] - 51s 259ms/step - loss: 0.0018 - accuracy: 0.6157 - val_loss: 0.0015 - val_accuracy: 0.5720
Epoch 2/50
195/195 [==============================] - 45s 231ms/step - loss: 0.0018 - accuracy: 0.6172 - val_loss: 0.0011 - val_accuracy: 0.5765
Epoch 3/50
195/195 [==============================] - 46s 234ms/step - loss: 0.0018 - accuracy: 0.6172 - val_loss: 0.0011 - val_accuracy: 0.5775
Epoch 4/50
195/195 [==============================] - 46s 234ms/step - loss: 0.0018 - accuracy: 0.6180 - val_loss: 0.0011 - val_accuracy: 0.5766
Epoch 5/50
195/195 [==============================] - 45s 231ms/step - loss: 0.0018 - accuracy: 0.6183 - val_loss: 0.0011 - val_accuracy: 0.5784
Epoch 6/50
195/195 [==============================] - 45s 232ms/step - loss: 0.0018 - accuracy: 0.6183 - val_loss: 0.0012 - val_accuracy: 0.5778
Epoch 7/50
195/195 [==============================] - 46s 235ms/step - loss: 0.0018 - accuracy: 0.6205 - val_loss: 0.0011 - val_ac

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,256,256,2] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2_1}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,256,256,2] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2_1}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[IteratorGetNext/_4]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored. [

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['accuracy'],label='training accuracy', color = "blue")
plt.plot(history.history['val_accuracy'],label='validation accuracy', color = "red")
plt.legend(loc=0)
plt.xlabel('epochs')
plt.ylabel('accuracies on dataset')
plt.grid(True)
plt.title("Training and validation accuracy")
plt.show()
fig.savefig('documentation\\images\\accuracy_CUDA.jpg')
plt.close(fig)

In [ ]:
# summarize history for loss
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'],label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.legend(loc=0)
plt.ylabel('losses on dataset')
plt.grid(True)
plt.title("Training and validation loss")
plt.show()
fig.savefig('documentation//images//loss_CUDA.jpg')
plt.close(fig)

In [ ]:
model_json = model.to_json()
with open('model66_CUDA.json', 'w') as json_file:
    json_file.write(model_json)

model.save('model66_CUDA.h5')

In [ ]:
# Load black and white images
color_me = []
for filename in os.listdir('test256/'):
        color_me.append(img_to_array(load_img('test256/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [ ]:
output = model.predict(color_me)
output = output * 128

In [ ]:
# Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    imsave("result_h66_CUDA/uint8_"+str(i)+".png", lab2rgb(result))

in_hsv_h = color.convert_colorspace(output, 'RGB', 'HSV')
in_hsv_s = in_hsv_h.copy()
in_hsv_v = in_hsv_h.copy()

for i in range(newImage.shape[0]):
    in_hsv_h[i,:,0] = np.sort(in_hsv_h[i,:,0])
    in_hsv_s[i,:,1] = np.sort(in_hsv_s[i,:,1])
    in_hsv_v[i,:,2] = np.sort(in_hsv_v[i,:,2])

imsave('testing-sorted-hue.png', (color.convert_colorspace(in_hsv_h, 'HSV', 'RGB')*255).astype(np.uint8))
imsave('testing-sorted-saturation.png', (color.convert_colorspace(in_hsv_s, 'HSV', 'RGB')*255).astype(np.uint8))